# Training CNN model

In [1]:
import pandas as pd
import numpy as np 
import os 
from numpy import savetxt, loadtxt 
import keras
from utils.KerasModels import generateExpData
from utils.KerasModels import buildCharCNNModel, buildCharGRUModel
from sklearn.metrics import balanced_accuracy_score, make_scorer, f1_score, precision_score, recall_score

keras.utils.set_random_seed(812)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-11-21 21:38:58.667445: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def prepare_data(data_directory):
    train = pd.read_csv(os.path.join(data_directory, 'train.csv'))
    validation = pd.read_csv(os.path.join(data_directory, 'validation.csv'))
    test = pd.read_csv(os.path.join(data_directory, 'test.csv'))

    dfTrainDataset = train[["screen_name", "text", "account.type"]]
    dfValDataset = validation[["screen_name", "text", "account.type"]]
    dfTestDataset = test[["screen_name", "text", "account.type"]]

    tokenizer = None
    train_features, tokenizer = generateExpData(dfTrainDataset, tokenizer = tokenizer)
    val_features, tokenizer = generateExpData(dfValDataset, tokenizer = tokenizer)
    test_features, tokenizer = generateExpData(dfTestDataset, tokenizer = tokenizer)

    dictLabels = {"human":0, "bot":1}
    y_train = dfTrainDataset["account.type"].apply(lambda x: dictLabels[x])
    y_val = dfValDataset["account.type"].apply(lambda x: dictLabels[x])
    y_test = dfTestDataset["account.type"].apply(lambda x: dictLabels[x])

    train_labels = y_train.tolist()
    val_labels = y_val.tolist()
    test_labels = y_test.tolist()

    vocab_size = len(tokenizer.word_index)

    return train_features, val_features, test_features, train_labels, val_labels, test_labels, vocab_size

def proba_to_pred(y_proba):
    y_pred_char_cnn = (y_proba > 0.5).astype(int)
    return y_pred_char_cnn

def calculate_metrics(y_true, y_pred):
    results = {
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred)
    }
    return results

In [4]:
DATA_PATH = os.path.join('..', 'data', 'preprocessed_url_simple')
train_features, val_features, test_features, train_labels, val_labels, test_labels, vocab_size = prepare_data(DATA_PATH)

model = buildCharCNNModel(vocab_size, embSize=32, inputSize=320)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 320)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 320, 32)      3360        ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 318, 128)     12416       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 317, 128)     16512       ['embedding[0][0]']              
                                                                                              

In [6]:
# Train CNN model.
model.fit(np.array(train_features), np.array(train_labels), batch_size=256, epochs=25, validation_data=(np.array(val_features), np.array(val_labels)))

Epoch 1/25
81/81 [==============================] - 19s 230ms/step - loss: 0.5495 - accuracy: 0.7490 - val_loss: 0.4694 - val_accuracy: 0.7685
Epoch 2/25
81/81 [==============================] - 19s 231ms/step - loss: 0.4283 - accuracy: 0.7972 - val_loss: 0.4291 - val_accuracy: 0.7880
Epoch 3/25
81/81 [==============================] - 19s 235ms/step - loss: 0.4000 - accuracy: 0.8079 - val_loss: 0.4032 - val_accuracy: 0.8102
Epoch 4/25
81/81 [==============================] - 19s 238ms/step - loss: 0.3827 - accuracy: 0.8186 - val_loss: 0.3912 - val_accuracy: 0.8167
Epoch 5/25
81/81 [==============================] - 20s 245ms/step - loss: 0.3719 - accuracy: 0.8234 - val_loss: 0.3828 - val_accuracy: 0.8210
Epoch 6/25
81/81 [==============================] - 20s 252ms/step - loss: 0.3622 - accuracy: 0.8289 - val_loss: 0.3746 - val_accuracy: 0.8310
Epoch 7/25
81/81 [==============================] - 19s 241ms/step - loss: 0.3529 - accuracy: 0.8341 - val_loss: 0.3724 - val_accuracy: 0.8306

In [7]:
probas = model.predict(np.array(test_features))
y_pred = proba_to_pred(probas)
results = calculate_metrics(test_labels, y_pred)
results

648/648 [==============================] - 6s 10ms/step


{'balanced_accuracy': 0.9210620142719128,
 'f1_score': 0.9217591041776332,
 'precision': 0.9134971070852699,
 'recall': 0.9301719142360441}

# Lemmatized data 

In [9]:
LEMMATIZED_DATA_PATH = os.path.join('..', 'data', 'lemmatized')
lem_train_features, lem_val_features, lem_test_features, lem_train_labels, lem_val_labels, lem_test_labels, lem_vocab_size = prepare_data(LEMMATIZED_DATA_PATH)

modelLEM = buildCharCNNModel(lem_vocab_size, embSize=32, inputSize=320, verbose=False)

In [10]:
# Train CNN model.
modelLEM.fit(np.array(lem_train_features), np.array(lem_train_labels), batch_size=256, epochs=25, validation_data=(np.array(lem_val_features), np.array(lem_val_labels)))

Epoch 1/25
81/81 [==============================] - 19s 222ms/step - loss: 0.5529 - accuracy: 0.7374 - val_loss: 0.4847 - val_accuracy: 0.7654
Epoch 2/25
81/81 [==============================] - 20s 241ms/step - loss: 0.4370 - accuracy: 0.7922 - val_loss: 0.4325 - val_accuracy: 0.7858
Epoch 3/25
81/81 [==============================] - 20s 245ms/step - loss: 0.4005 - accuracy: 0.8088 - val_loss: 0.4032 - val_accuracy: 0.8067
Epoch 4/25
81/81 [==============================] - 20s 251ms/step - loss: 0.3779 - accuracy: 0.8199 - val_loss: 0.3926 - val_accuracy: 0.8232
Epoch 5/25
81/81 [==============================] - 21s 265ms/step - loss: 0.3688 - accuracy: 0.8271 - val_loss: 0.3793 - val_accuracy: 0.8223
Epoch 6/25
81/81 [==============================] - 21s 260ms/step - loss: 0.3590 - accuracy: 0.8315 - val_loss: 0.3778 - val_accuracy: 0.8189
Epoch 7/25
81/81 [==============================] - 21s 258ms/step - loss: 0.3504 - accuracy: 0.8350 - val_loss: 0.3680 - val_accuracy: 0.8328

In [11]:
probas_lem = modelLEM.predict(np.array(lem_test_features))
y_pred_lem = proba_to_pred(probas_lem)
results_lem = calculate_metrics(lem_test_labels, y_pred_lem)
results_lem

648/648 [==============================] - 5s 8ms/step


{'balanced_accuracy': 0.9082764405352697,
 'f1_score': 0.9130673499267936,
 'precision': 0.8675013041210224,
 'recall': 0.9636855321614834}

# Stemmed data

In [12]:
STEMMED_DATA_PATH = os.path.join('..', 'data', 'stemmed')
stem_train_features, stem_val_features, stem_test_features, stem_train_labels, stem_val_labels, stem_test_labels, stem_vocab_size = prepare_data(STEMMED_DATA_PATH)

modelSTEM = buildCharCNNModel(stem_vocab_size, embSize=32, inputSize=320, verbose=False)

In [13]:
# Train CNN model.
modelSTEM.fit(np.array(stem_train_features), np.array(stem_train_labels), batch_size=256, epochs=25, validation_data=(np.array(stem_val_features), np.array(stem_val_labels)))

Epoch 1/25
81/81 [==============================] - 19s 226ms/step - loss: 0.5619 - accuracy: 0.7349 - val_loss: 0.4777 - val_accuracy: 0.7680
Epoch 2/25
81/81 [==============================] - 17s 210ms/step - loss: 0.4320 - accuracy: 0.7950 - val_loss: 0.4283 - val_accuracy: 0.7937
Epoch 3/25
81/81 [==============================] - 19s 235ms/step - loss: 0.3983 - accuracy: 0.8107 - val_loss: 0.4026 - val_accuracy: 0.8071
Epoch 4/25
81/81 [==============================] - 19s 236ms/step - loss: 0.3806 - accuracy: 0.8206 - val_loss: 0.3915 - val_accuracy: 0.8219
Epoch 5/25
81/81 [==============================] - 19s 237ms/step - loss: 0.3673 - accuracy: 0.8256 - val_loss: 0.3810 - val_accuracy: 0.8271
Epoch 6/25
81/81 [==============================] - 19s 239ms/step - loss: 0.3586 - accuracy: 0.8284 - val_loss: 0.3780 - val_accuracy: 0.8315
Epoch 7/25
81/81 [==============================] - 20s 244ms/step - loss: 0.3534 - accuracy: 0.8325 - val_loss: 0.3692 - val_accuracy: 0.8345

In [14]:
probas_stem = modelSTEM.predict(np.array(stem_test_features))
y_pred_stem = proba_to_pred(probas_stem)
results_stem = calculate_metrics(stem_test_labels, y_pred_stem)
results_stem

648/648 [==============================] - 6s 9ms/step


{'balanced_accuracy': 0.9185041834188477,
 'f1_score': 0.9196649533599848,
 'precision': 0.9065490711202853,
 'recall': 0.9331659262120919}

In [19]:
# Create csv with all results

results_all = pd.DataFrame([results, results_lem, results_stem])
results_all['model'] = 'CharCNN'
results_all['dataset'] = ['simple', 'lemmatized', 'stemmed']

if not os.path.exists('results'):
    os.makedirs('results')
results_all.to_csv('results/char_cnn.csv', index=False)
results_all

,balanced_accuracy,f1_score,precision,recall,model,dataset
0,0.921062,0.921759,0.913497,0.930172,CharCNN,simple
1,0.908276,0.913067,0.867501,0.963686,CharCNN,lemmatized
2,0.918504,0.919665,0.906549,0.933166,CharCNN,stemmed
